In [1]:
import flodym as fd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from flodym import StockArray

In [2]:
time_pv = fd.Dimension(name="Time", letter="t", items=list(range(2025, 2046)))
dims_pv = fd.DimensionSet(dim_list=[time_pv])

time_batt = fd.Dimension(name="Time", letter="t", items=list(range(2025, 2046)))
dims_batt = fd.DimensionSet(dim_list=[time_batt])

In [3]:
PV = np.array([110,131,152,173,194,215,234,253,272,291,310,328,346,364,382,400,416,432,448,464,480])

Batt = np.array([27.50,37.17,48.25,60.75,74.67,90.00,104.10,119.20,135.30,152.40,170.50,190.24,211.06,232.96,255.94,280.00,303.68,328.32,353.92,380.48,408.00
])

In [4]:
PV_stock_GWp = StockArray(dims=dims_pv)
PV_stock_GWp.values[...] = PV

Batt_stock_GWh = StockArray(dims=dims_batt)
Batt_stock_GWh.values[...] = Batt


In [5]:
lifetime_pv = fd.lifetime_models.LogNormalLifetime(dims=dims_pv, mean=20, std=4)
lifetime_batt = fd.lifetime_models.LogNormalLifetime(dims=dims_batt, mean=10, std=2)

In [6]:
dsm_pv = fd.StockDrivenDSM(
    dims=dims_pv, 
    lifetime_model=lifetime_pv,
    stock=PV_stock_GWp
)
dsm_batt = fd.StockDrivenDSM(
    dims=dims_batt, 
    lifetime_model=lifetime_batt,
    stock=Batt_stock_GWh
)

In [7]:
dsm_pv.compute()
dsm_batt.compute()

In [ ]:
## COMBINED MODEL: DOES NOT WORK####
# time = fd.Dimension(name="Time", letter="t", items=list(range(2025, 2046)))
# product = fd.Dimension(name = "Product", letter = "p", items=["PV", "Battery"])
# dims = fd.DimensionSet(dim_list=[time, product])

# Stock_total =StockArray(dims=dims)
# Stock_total["PV"].values[...] = PV
# Stock_total["Battery"].values[...] = Batt
# print(Stock_total.to_df())

# PV_stock_GWp =StockArray..from_df(dims=dims_pv, df=PV)
# Batt_stock_GWh =StockArray.from_df(dims=dims_batt, df=Batt)


# lifetime_mean = fd.FlodymArray(dims=dims["p",])
# lifetime_mean["PV"].values[...] = 20.
# lifetime_mean["Battery"].values[...] = 10.
    
# lifetime_std = fd.FlodymArray(dims=dims["p",])
# lifetime_std["PV"].values[...] = 4.
# lifetime_std["Battery"].values[...] = 2.
    
# lifetime = fd.lifetime_models.LogNormalLifetime(dims=dims, mean=lifetime_mean, std=lifetime_std)

# dsm = fd.StockDrivenDSM(
#     dims=dims, 
#     lifetime_model=lifetime,
#     stock=Stock_total
# )
# dsm.compute()

In [8]:
print("\n PV RESULTS")

print("\n--- Stock (should match input) ---")
print(dsm_pv.stock.to_df())

print("\n--- Inflow (new installations needed) ---")
print(dsm_pv.inflow.to_df())

print("\n--- Outflow (end-of-life waste) ---")
print(dsm_pv.outflow.to_df())

print("\n BATTERY RESULTS")
print("\n--- Stock (should match input) ---")
print(dsm_batt.stock.to_df())

print("\n--- Inflow (new installations needed) ---")
print(dsm_batt.inflow.to_df())

print("\n--- Outflow (end-of-life waste) ---")
print(dsm_batt.outflow.to_df())


print("VERIFICATION")

# Check: Stock balance should hold: Stock[t] = Stock[t-1] + Inflow[t] - Outflow[t]
pv_balance_check = (
    dsm_pv.stock.values[1:] - dsm_pv.stock.values[:-1] -  dsm_pv.inflow.values[1:] + dsm_pv.outflow.values[1:])
print(f"\nPV Stock balance error (should be ~0): {np.abs(pv_balance_check).max():.6f}")

batt_balance_check = (
    dsm_batt.stock.values[1:] - dsm_batt.stock.values[:-1] - 
    dsm_batt.inflow.values[1:] + dsm_batt.outflow.values[1:]
)
print(f"Battery Stock balance error (should be ~0): {np.abs(batt_balance_check).max():.6f}")

# Summary statistics
print(f"\n--- PV Summary ---")
print(f"Total inflow (2025-2045): {dsm_pv.inflow.values.sum():.1f} GWp")
print(f"Total outflow (2025-2045): {dsm_pv.outflow.values.sum():.1f} GWp")
print(f"Final stock (2045): {dsm_pv.stock.values[-1]:.1f} GWp")

print(f"\n--- Battery Summary ---")
print(f"Total inflow (2025-2045): {dsm_batt.inflow.values.sum():.1f} GWh")
print(f"Total outflow (2025-2045): {dsm_batt.outflow.values.sum():.1f} GWh")
print(f"Final stock (2045): {dsm_batt.stock.values[-1]:.1f} GWh")


 PV RESULTS

--- Stock (should match input) ---
      value
Time       
2025  110.0
2026  131.0
2027  152.0
2028  173.0
2029  194.0
2030  215.0
2031  234.0
2032  253.0
2033  272.0
2034  291.0
2035  310.0
2036  328.0
2037  346.0
2038  364.0
2039  382.0
2040  400.0
2041  416.0
2042  432.0
2043  448.0
2044  464.0
2045  480.0

--- Inflow (new installations needed) ---
           value
Time            
2025  110.000000
2026   21.000000
2027   21.000000
2028   21.000000
2029   21.000000
2030   21.000000
2031   19.000001
2032   19.000066
2033   19.001280
2034   19.012794
2035   19.077166
2036   18.315321
2037   18.949386
2038   20.242668
2039   22.361165
2040   25.249403
2041   26.615041
2042   30.030856
2043   33.083478
2044   35.488182
2045   37.131534

--- Outflow (end-of-life waste) ---
             value
Time              
2025  0.000000e+00
2026  0.000000e+00
2027  0.000000e+00
2028  0.000000e+00
2029  5.837553e-12
2030  7.504580e-09
2031  1.346110e-06
2032  6.558589e-05
2033  1.280185